In [ ]:
# Import Required Libraries
import backtrader as bt
import backtrader.feeds as btfeeds
import pandas as pd
import datetime
import numpy as np

print("Libraries imported successfully!")
print("✓ Backtrader framework ready")
print("✓ Pandas for data handling")

In [ ]:
# Strategy000Simplified - Practical Implementation

class Strategy000Simplified(bt.Strategy):
    """
    Simplified version focusing on MACD crossover with relaxed Bulls Power conditions
    Uses practical Williams %R levels for exits instead of impossible -100 level
    """
    
    params = (
        ('macd_fast', 24),
        ('macd_slow', 54), 
        ('macd_signal', 14),
        ('bulls_power_1_period', 12),
        ('bulls_power_2_period', 27),
        ('williams_period', 55),
        ('williams_level', -80.0),  # Practical oversold level
        ('printlog', False),
    )

    def __init__(self):
        self.macd_ind = bt.indicators.MACD(
            self.data.close,
            period_me1=self.p.macd_fast,
            period_me2=self.p.macd_slow,
            period_signal=self.p.macd_signal
        )
        self.macd = self.macd_ind.macd
        self.macd_signal = self.macd_ind.signal
        self.macd_diff = self.macd - self.macd_signal
        
        # Bulls Power 
        self.ema_12 = bt.indicators.EMA(self.data.close, period=self.p.bulls_power_1_period)
        self.bulls_power_1 = self.data.high - self.ema_12
        
        self.ema_27 = bt.indicators.EMA(self.data.close, period=self.p.bulls_power_2_period)  
        self.bulls_power_2 = self.data.high - self.ema_27
        
        # Williams %R
        self.williams_pr = bt.indicators.WilliamsR(self.data, period=self.p.williams_period)
        
        self.order = None
        self.trade_count = 0

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()}, {txt}')

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}')
            else:
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}')
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.trade_count += 1
        self.log(f'TRADE #{self.trade_count}: PROFIT {trade.pnl:.2f}', doprint=True)

    def next(self):
        if len(self.data) < 60:  # Wait for indicators to stabilize
            return

        if self.order:
            return

        if not self.position:
            # Entry: MACD crossover + Bulls Power positive
            try:
                macd_diff_current = self.macd_diff[0]
                macd_diff_prev = self.macd_diff[-1]
                
                # MACD crossover
                macd_bull_cross = (macd_diff_current > 0 and macd_diff_prev <= 0)
                macd_bear_cross = (macd_diff_current < 0 and macd_diff_prev >= 0)
                
                # Bulls Power conditions - simplified to just be positive
                bulls_1_positive = self.bulls_power_1[0] > 0
                bulls_2_positive = self.bulls_power_2[0] > 0
                
                if macd_bull_cross and bulls_1_positive and bulls_2_positive:
                    self.log(f'BUY: MACD cross, Bulls1={self.bulls_power_1[0]:.2f}, Bulls2={self.bulls_power_2[0]:.2f}')
                    self.order = self.buy()
                    
                elif macd_bear_cross:  # Short on bear cross regardless of Bulls Power
                    self.log(f'SELL: MACD bear cross')
                    self.order = self.sell()
                    
            except (IndexError, TypeError):
                return
        else:
            # Exit on Williams %R extremes
            try:
                williams = self.williams_pr[0]
                
                # Exit long positions when Williams %R becomes overbought
                if self.position.size > 0 and williams > -20:  # Overbought
                    self.log(f'EXIT LONG: Williams overbought {williams:.1f}')
                    self.order = self.sell()
                    
                # Exit short positions when Williams %R becomes oversold  
                elif self.position.size < 0 and williams < self.p.williams_level:  # Oversold
                    self.log(f'EXIT SHORT: Williams oversold {williams:.1f}')
                    self.order = self.buy()
                    
            except (IndexError, TypeError):
                return

    def stop(self):
        self.log(f'Strategy Simplified: {self.trade_count} trades, Final Value: ${self.broker.getvalue():.2f}', doprint=True)

print("✓ Strategy000Simplified created - practical implementation with realistic exit levels!")

In [ ]:
# Backtest Function for Strategy000Simplified

def test_simplified_strategy():
    """Test the simplified strategy with S&P 500 data"""
    print("=== TESTING STRATEGY000SIMPLIFIED ===\n")
    
    cerebro = bt.Cerebro()
    
    # Load S&P 500 data
    spy_file = r'../04_S&P500_quant_analysis/01_data/S&P500_D_1789-05-01_2025-09-17.csv'
    df = pd.read_csv(spy_file, skiprows=3, header=None)
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date').dropna()
    
    # Use last 5 years for testing
    end_date = df['Date'].max()
    start_date = end_date - pd.DateOffset(years=5)
    df_recent = df[df['Date'] >= start_date].copy()
    
    print(f"📊 Data: {len(df_recent)} trading days from {start_date.date()} to {end_date.date()}")
    
    # Create temporary file for backtrader
    temp_file = 'temp_simplified_test.csv'
    df_recent.to_csv(temp_file, index=False)
    
    # Create data feed
    data = btfeeds.GenericCSVData(
        dataname=temp_file,
        dtformat=('%Y-%m-%d'),
        datetime=0, open=1, high=2, low=3, close=4, volume=5,
        openinterest=-1, headers=True,
    )

    # Add data and strategy
    cerebro.adddata(data)
    cerebro.addstrategy(Strategy000Simplified, printlog=False)
    
    # Set initial capital and commission
    initial_capital = 100000.0
    cerebro.broker.setcash(initial_capital)
    cerebro.broker.setcommission(commission=0.001)  # 0.1% commission
    
    # Add analyzers
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="trades")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    print(f"💰 Starting Capital: ${initial_capital:,.2f}")
    print("🚀 Running backtest...\n")
    
    # Run backtest
    results = cerebro.run()
    
    # Clean up temporary file
    import os
    if os.path.exists(temp_file):
        os.remove(temp_file)
    
    # Calculate results
    final_value = cerebro.broker.getvalue()
    total_return = ((final_value - initial_capital) / initial_capital) * 100
    
    print(f"\n=== BACKTEST RESULTS ===")
    print(f"💰 Final Portfolio Value: ${final_value:,.2f}")
    print(f"📈 Total Return: {total_return:.2f}%")
    print(f"💵 Net Profit: ${final_value - initial_capital:,.2f}")
    
    # Analyze trades
    strat = results[0]
    trade_analysis = strat.analyzers.trades.get_analysis()
    
    if trade_analysis and hasattr(trade_analysis, 'total'):
        total_trades = getattr(trade_analysis.total, 'total', 0)
        print(f"\n=== TRADE ANALYSIS ===")
        print(f"📊 Total Trades: {total_trades}")
        
        # Winning trades
        if hasattr(trade_analysis, 'won'):
            won_trades = getattr(trade_analysis.won, 'total', 0)
            won_pnl = getattr(trade_analysis.won, 'pnl', {}).get('total', 0)
            win_rate = (won_trades / total_trades * 100) if total_trades > 0 else 0
            print(f"✅ Winning Trades: {won_trades} ({win_rate:.1f}%)")
            if won_pnl:
                print(f"💰 Winning PnL: ${won_pnl:.2f}")
        
        # Losing trades  
        if hasattr(trade_analysis, 'lost'):
            lost_trades = getattr(trade_analysis.lost, 'total', 0)
            lost_pnl = getattr(trade_analysis.lost, 'pnl', {}).get('total', 0)
            print(f"❌ Losing Trades: {lost_trades}")
            if lost_pnl:
                print(f"💸 Losing PnL: ${lost_pnl:.2f}")
    
    # Performance metrics
    print(f"\n=== PERFORMANCE METRICS ===")
    
    # Sharpe Ratio
    sharpe_analysis = strat.analyzers.sharpe.get_analysis()
    if sharpe_analysis and 'sharperatio' in sharpe_analysis:
        sharpe_ratio = sharpe_analysis['sharperatio'] or 0
        print(f"📊 Sharpe Ratio: {sharpe_ratio:.4f}")

    # Drawdown Analysis
    drawdown_analysis = strat.analyzers.drawdown.get_analysis()
    if drawdown_analysis:
        max_dd = drawdown_analysis.get('max', {}).get('drawdown', 0)
        print(f"📉 Max Drawdown: {max_dd:.2f}%")
    
    return results, total_trades, final_value

print("✓ Backtest function ready for Strategy000Simplified")

In [ ]:
# Execute Strategy000Simplified Backtest
try:
    results, trade_count, final_value = test_simplified_strategy()
    
    print(f"\n🎉 BACKTEST COMPLETED SUCCESSFULLY!")
    print(f"📊 Summary: {trade_count} trades, Final Value: ${final_value:,.2f}")
    
    if trade_count > 10:
        print("✅ Strategy shows active trading with multiple trades")
    elif trade_count > 1:
        print("📈 Strategy executed multiple trades")
    else:
        print("⚠️ Strategy was very selective with few trades")
        
except Exception as e:
    print(f"❌ Error running backtest: {str(e)}")
    print("💡 Make sure the S&P 500 data file exists at the specified path")